In [1]:
import numpy as np
from scipy.spatial.distance import cosine as cosdist
import csv

# Apadrinhamatch

Código que pega respostas de um Google Forms e trata-as de acordo, gerando um perfil para cada usuário que respondeu. Depois, é feito um "match" entre calouros e veteranos com perfis mais próximos (calculando a distância de coseno entre os vetores de cada usuário).

O vetor perfil (e as pontuações para cada resposta) foi gerado de maneira completamente arbitrária, e modificações podem ser feitas nele para melhores resultados.

Ordem dos elementos do vetor do aluno:
- Nome
- Telefone
- Calouro/Veterano
- Onde mora
- Interesses acadêmicos
- Rolês
- Hobbies
- Outras áreas
- Período
- Barra
- Jacarepaguá
- Centro
- Grande Tijuca
- Ilha/Alojamento
- Niterói
- Zona Sul
- Zona Norte
- Zona Oeste
- Baixada Fluminense

In [2]:
# Funcao que pega as letras de cada resposta
def getOptions(opcoes):
    temp = opcoes.split(";")
    
    if (temp == ['']):
        return []
    
    return [x[0] for x in temp]

# Instanciando os vetores de calouros e veteranos
calouros = []
veteranos = []

# Fazendo o parse dos alunos do csv 
with open('respostas.csv', newline='') as f:
    reader = csv.reader(f)
    primeira = True
    
    for row in reader:
        # Ignoramos a primeira linha do arquivo
        if (primeira):
            primeira = False
            continue
            
        # Instanciando aluno 
        aluno = {"Nome": "", "Vetor": np.zeros (26, dtype = int), "Telefone": "", "Casta": "",
                 "Local": "", "Interesses": [], "Roles": [], "Hobbies": [], "Areas": []}
        
        # Incluindo as informacoes contidas na linha
        aluno["Nome"] = row[1]
        aluno["Telefone"] = row[2]
        aluno["Casta"] = row[3]
        aluno["Local"] = getOptions(row[4]) 
        aluno["Interesses"] = getOptions(row[5])
        aluno["Roles"] = getOptions(row[6])
        aluno["Hobbies"] = getOptions(row[7])
        aluno["Areas"] = getOptions(row[8])
        
        # Passando o aluno pra lista correspondente, junto com as informacoes exclusivas
        # de cada "casta". Calouros possuem uma lista com a pontuacao de cada vet, e veteranos
        # guardam o periodo em que estao
        if (aluno["Casta"] == "Calouro"):
            aluno["Ranking"] = []
            calouros.append(aluno.copy())
        else:
            aluno["Periodo"] = getOptions(row[9])
            veteranos.append(aluno.copy())

## Dicionários de vetores para as respostas

In [3]:
# Dicionario de vetores para as respostas de onde mora
locais = {"a": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 1, 1]),
          "b": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 1]),
          "c": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0]),
          "d": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 0]),
          "e": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]),
          "f": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
          "g": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0]),
          "h": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0]),
          "i": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 2, 0]),
          "j": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 2])}

In [4]:
# Dicionario de vetores para os interesses extracurriculares
interesses = {"a": np.array([0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
              "b": np.array([0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
              "c": np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
              "d": np.array([2, 2, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, -1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
              "e": np.array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
              "f": np.array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
              "g": np.array([0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
              "h": np.array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [5]:
# Dicionario de vetores para os roles que mais interessam
roles = {"a": np.array([0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "b": np.array([0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "c": np.array([0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, -1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "d": np.array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "e": np.array([0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "f": np.array([0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "g": np.array([0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "h": np.array([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "i": np.array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "j": np.array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "k": np.array([0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [6]:
# Dicionario de vetores para os hobbies que mais interessam
hobbies = {"a": np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
           "b": np.array([0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
           "c": np.array([0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
           "d": np.array([-1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
           "e": np.array([1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
           "f": np.array([0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
           "g": np.array([0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
           "h": np.array([2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
           "i": np.array([2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
           "j": np.array([0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 2, -1, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
           "k": np.array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
           "l": np.array([-1, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [7]:
# Dicionario de vetores para as outras areas disciplinares
areas = {"a": np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "b": np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "c": np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "d": np.array([0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "e": np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "f": np.array([0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "g": np.array([0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "h": np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
         "i": np.array([0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [8]:
# Dicionario de vetores para o periodo em que esta
periodo = {"a": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
           "b": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),   
           "c": np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

## Cálculo dos vetores de cada aluno

In [9]:
# Geracao dos vetores de cada aluno
for aluno in veteranos + calouros:
    # Resposta de onde mora
    if (len(aluno["Local"]) != 0):
        aluno["Vetor"] += locais[aluno["Local"][0]]

    # Resposta de interesses extracurriculares
    for opcoes in aluno["Interesses"]:
        aluno["Vetor"] += interesses[opcoes]

    # Resposta de roles
    for opcoes in aluno["Roles"]:
        aluno["Vetor"] += roles[opcoes]

    # Respostas de hobbies
    for opcoes in aluno["Hobbies"]:
        aluno["Vetor"] += hobbies[opcoes]

    # Respostas para outras areas
    for opcoes in aluno["Areas"]:
        aluno["Vetor"] += areas[opcoes]

    # Respostas para periodos
    if (aluno["Casta"] == "Veterano"):
        aluno["Vetor"] += periodo[aluno["Periodo"][0]]
    else:
        aluno["Vetor"] += periodo["b"]

{'Nome': 'aaa', 'Vetor': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0]), 'Telefone': 'aaa', 'Casta': 'Veterano', 'Local': ['a'], 'Interesses': ['b', 'c', 'd'], 'Roles': ['f', 'h'], 'Hobbies': ['e', 'i'], 'Areas': ['c', 'g'], 'Periodo': ['a']}
{'Nome': 'Juliana Nub', 'Vetor': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0]), 'Telefone': '(21) Soumuito-nub!', 'Casta': 'Veterano', 'Local': ['j'], 'Interesses': ['d', 'f', 'g'], 'Roles': ['c', 'e', 'i'], 'Hobbies': ['a'], 'Areas': ['g'], 'Periodo': ['c']}
{'Nome': 'Juliana', 'Vetor': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0]), 'Telefone': '9999999999999', 'Casta': 'Veterano', 'Local': ['g'], 'Interesses': ['a'], 'Roles': ['d'], 'Hobbies': ['a', 'j', 'k', 'l'], 'Areas': ['a', 'd', 'f'], 'Periodo': ['b']}
{'Nome': 'Ada Lovelace', 'Vetor': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## Cálculo das distâncias e atribuição dos pares

In [10]:
# Calculando a distancia do coseno entre cada calouro e os veteranos
for cal in calouros:
    # "Reiniciamos" o vetor para podermos rodar essa celula varias vezes
    cal["Ranking"] = []
    
    # Calculamos a distancia, veterano a veterano, e armazenamos em "Ranking"
    # como [veterano, distancia]
    for vet in veteranos:
        dist = cosdist(cal["Vetor"], vet["Vetor"])
        cal["Ranking"].append([vet, dist])

    # Ordenamos a lista
    cal["Ranking"].sort(key=lambda x: x[1])

In [11]:
# PARA DEBUG
#for cal in calouros:
#  print(cal["Nome"])
#  for entry in cal["Ranking"]:
#    print(entry[0]["Nome"] + " " + str(entry[1]))
#  print()

In [12]:
# O vetor de resultados sera mais enxuto
resultados = [["Calouro", "Contato", "Veterano", "Contato"]]

# Percorremos a lista calouros ate que ela fique vazia
while (len(calouros) > 0):
    # Pegamos o primeiro calouro da lista, e seu veterano correspondente
    calFinal = calouros[0]
    vetFinal = calFinal["Ranking"][0][0]

    # Para cada outro calouro na lista... 
    for cal in calouros[1:]:
        # Se o veterano nao for o "vitorioso" para cal, o removemos do Ranking
        if (cal["Ranking"][0][0] != vetFinal):
            cal["Ranking"] = [x for x in cal["Ranking"] if x[0] != vetFinal]
        else:
            # Se for, vemos que calouro esta mais perto dele, que sera o calouro
            # atribuido ao veterano 
            if (cal ["Ranking"][0][1] < calFinal ["Ranking"][0][1]):
                indice = calouros.index(calFinal)
                calouros[indice]["Ranking"].pop(0)
                calFinal = cal
            else:
                cal["Ranking"].pop(0)

    # Adicionamos o par calouro/veterano a lista final e removemos o calFinal de calouros
    resultados.append([calFinal["Nome"], calFinal["Telefone"], vetFinal["Nome"], vetFinal["Telefone"]])
    calouros.remove(calFinal)

In [13]:
resultados

[['Calouro', 'Contato', 'Veterano', 'Contato'],
 ['teste', 'teste', 'Juliana', '9999999999999'],
 ['Ana Hickmann', 'Bom Dia', 'Ada Lovelace', '190'],
 ['Felipe Prior', '(21) Babu-Santana', 'Who let the dogs out', 'who who who'],
 ['matheus', 'matheus', 'aaa', 'aaa']]

In [14]:
# Exportando os resultados para um csv
with open('resultados.csv', 'w', newline='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerows(resultados)